<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Full_Training_%26_Evaluation_Script_with_Differential_Privacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opacus

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from opacus import PrivacyEngine

# --- Simple Neural Network ---
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

# --- Configuration ---
batch_size = 64
learning_rate = 0.1
epochs = 5
delta = 1e-5
target_epsilon = 5.0
max_grad_norm = 1.0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Data Loading ---
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))  # Flatten to 784
])

train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

# --- Model, Optimizer, Criterion ---
model = SimpleNN().to(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# --- Privacy Engine ---
privacy_engine = PrivacyEngine()

model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    target_epsilon=target_epsilon,
    target_delta=delta,
    epochs=epochs,
    max_grad_norm=max_grad_norm,
)

# --- Training Loop ---
for epoch in range(1, epochs + 1):
    model.train()
    total_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    epsilon = privacy_engine.get_epsilon(delta)
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch}/{epochs} | Loss: {avg_loss:.4f} | ε = {epsilon:.2f}")

print("✅ Training complete!")

# --- Evaluation ---
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

accuracy = 100.0 * correct / total
print(f"\n🧪 Test Accuracy: {accuracy:.2f}%")